In [33]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [34]:
import xgboost

In [35]:
from xgboost import XGBClassifier

In [36]:
xgboost.__version__

'1.7.4'

In [37]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

G2_TPM_merged_lats = pd.read_csv('../../G2Surface/G2_surface_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

,PF00003,PF00013,PF00022,PF00034,PF00071,PF00124,PF00125,PF00139,PF00169,PF00185,...,PF14259,PF14727,PF14778,PF14782,PF14870,PF15104,PF15782,PF16158,PF16313,PF16672
0,0.0,408.086895,252.912087,4.379237,285.889979,0.000000,239.290884,0.0,133.952240,19.947464,...,0,15.853548,0.0,0.610060,0.000000,0.000000,0,0.000000,0.000000,0.0
1,0.0,650.060793,123.075230,12.973424,571.339674,0.000000,102.135187,0.0,31.378286,13.236538,...,0,54.369949,0.0,0.000000,0.000000,31.366027,0,11.511522,48.602594,0.0
2,0.0,618.487663,218.850359,0.000000,547.281282,0.000000,137.779083,0.0,38.001376,0.000000,...,0,0.000000,0.0,0.000000,39.335042,22.600627,0,7.597548,0.000000,0.0
3,0.0,399.637051,136.609097,0.000000,167.185325,0.000000,0.000000,0.0,89.359341,0.000000,...,0,0.000000,0.0,0.000000,0.000000,0.000000,0,24.152300,5.926447,0.0
4,0.0,303.928060,246.424777,0.000000,327.109367,0.000000,292.478920,0.0,131.137501,18.286421,...,0,2.270390,0.0,0.524201,4.835170,1.482803,0,0.000000,6.497064,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.0,478.588219,65.285098,0.000000,203.035206,0.000000,0.000000,0.0,58.491910,2.214429,...,0,0.000000,0.0,45.920952,0.000000,0.000000,0,5.409240,4.261426,0.0
318,0.0,367.911190,304.669865,2.381515,296.744936,0.000000,297.677523,0.0,126.146757,45.288425,...,0,0.000000,0.0,2.365290,3.070624,5.377761,0,1.795717,7.103138,0.0
319,0.0,0.000000,3549.267886,882.576563,4226.603938,0.000000,137.584507,0.0,624.479815,0.000000,...,0,0.000000,0.0,0.000000,0.000000,50.618799,0,129.250044,0.000000,0.0
320,0.0,807.011216,257.890369,0.000000,451.908531,0.000000,142.376623,0.0,73.340492,25.665818,...,0,24.479300,0.0,9.487224,4.556036,3.489262,0,43.403320,85.969743,0.0


In [38]:
G2_TPM_merged_lats.columns[(G2_TPM_merged_lats == 0).all()]

Index(['PF07784', 'PF08713', 'PF14259', 'PF15782'], dtype='object')

In [41]:
features = features['pfam']

0      PF09286
1      PF02514
2      PF13925
3      PF04133
4      PF03645
        ...   
178    PF05148
179    PF04922
180    PF01026
181    PF00588
182    PF00034
Name: pfam, Length: 183, dtype: object

In [48]:
len(features)

183

In [49]:
G2_TPM_merged_lats = G2_TPM_merged_lats[features]

In [51]:
G2_TPM_merged_lats.columns[(G2_TPM_merged_lats == 0).all()]

Index(['PF14259', 'PF08713', 'PF15782', 'PF07784'], dtype='object')

In [52]:
train_data = train_data[features]

In [53]:
le = LabelEncoder()

In [54]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [55]:
G2 = scaler.transform(G2_TPM_merged_lats)

In [56]:
#gamma: 0.5, learning_rate: 0.2, max_depth: 3, n_estimators: 100, reg_lambda: 0.0 
model = XGBClassifier(gamma=0.5, learning_rate=0.2, max_depth=3, n_estimators=100, reg_lambda=0.0)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [57]:
xg_predictions_G2 = model.predict(G2)

In [59]:
prob = model.predict_proba(G2)

In [60]:
G2_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_G2})

In [61]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [63]:
G2_predictions.to_csv('../../g2Surface/G2_surface_trophicModePredictions_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)

In [64]:
probDF.to_csv('../../g2Surface/G2_trophicModePredictionsProbabilities_surface_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)